In [1]:
import os
import re
import urllib.request
from importlib.metadata import version
from dataclasses import dataclass

try:
    import tiktoken
except ImportError:
    !pip install tiktoken
    import tiktoken

try:
    import torch
    import torch.nn as nn
except ImportError:
    !pip install torch
    import torch
    import torch.nn as nn

print("torch version:", torch.__version__)
print("tiktoken version:", version("tiktoken"))

torch version: 2.1.2+cpu
tiktoken version: 0.5.2


In [2]:
from utils.attentions import MultiHeadAttention

## GPT2-124M
- "vocab_size" indicates a vocabulary size of 50,257 words, supported by the BPE tokenizer discussed in Chapter 2
- "context_length" represents the model's maximum input token count, as enabled by positional embeddings covered in Chapter 2
- "emb_dim" is the embedding size for token inputs, converting each input token into a 768-dimensional vector
- "n_heads" is the number of attention heads in the multi-head attention mechanism implemented in Chapter 3
- "n_layers" is the number of transformer blocks within the model, which we'll implement in upcoming sections
- "drop_rate" is the dropout mechanism's intensity, discussed in Chapter 3; 0.1 means dropping 10% of hidden units during training to mitigate overfitting
- "qkv_bias" decides if the Linear layers in the multi-head attention mechanism (from Chapter 3) should include a bias vector when computing query (Q), key (K), and value (V) tensors; we'll disable this option, which is standard practice in modern LLMs; however, we'll revisit this later when loading pretrained GPT-2 weights from OpenAI into our reimplementation in chapter 5

- **GPT2-small (the 124M configuration we already implemented):**

- "emb_dim" = 768
- "n_layers" = 12
- "n_heads" = 12
- **GPT2-medium:**

- "emb_dim" = 1024
- "n_layers" = 24
- "n_heads" = 16
- **PT2-large:**

- "emb_dim" = 1280
- "n_layers" = 36
- "n_heads" = 20
- **GPT2-XL:**

- "emb_dim" = 1600
- "n_layers" = 48
- "n_heads" = 25

In [3]:
@dataclass
class Params:
    vocab_size= 50257   # Vocabulary size
    context_length= 1024 # Context length
    emb_dim= 768         # Embedding dimension
    n_heads= 12          # Number of attention heads
    n_layers= 12         # Number of layers
    drop_rate= 0.1       # Dropout rate
    qkv_bias= False       # Query-Key-Value bias

my_params = Params()

In [4]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [5]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [6]:
class FeedForward(nn.Module):
    def __init__(self, params: Params):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(params.emb_dim, 4 * params.emb_dim),
            GELU(),
            nn.Linear(4 * params.emb_dim, params.emb_dim),
        )

    def forward(self, x):
        return self.layers(x)

In [7]:
class TransformerBlock(nn.Module):
    def __init__(self, params: Params):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=params.emb_dim,
            d_out=params.emb_dim,
            context_length=params.context_length,
            num_heads=params.n_heads, 
            dropout=params.drop_rate,
            qkv_bias=params.qkv_bias
        )
        self.ff = FeedForward(params)
        self.norm1 = LayerNorm(params.emb_dim)
        self.norm2 = LayerNorm(params.emb_dim)
        self.drop_shortcut = nn.Dropout(params.drop_rate)

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [8]:
class GPTModel(nn.Module):
    def __init__(self, params: Params):
        super().__init__()
        self.tok_emb = nn.Embedding(params.vocab_size, params.emb_dim)
        self.pos_emb = nn.Embedding(params.context_length, params.emb_dim)
        self.drop_emb = nn.Dropout(params.drop_rate)
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(params) for _ in range(params.n_layers)])
        
        self.final_norm = LayerNorm(params.emb_dim)
        self.out_head = nn.Linear(params.emb_dim, params.vocab_size, bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [9]:
torch.manual_seed(123)

# create 2 training examples with 5 dimensions (features) each
tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)

# initialize GPT model
model = GPTModel(my_params)
out = model(batch)

print(f"Input text: \n {txt1} \n {txt2}")
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
# print(out)

Input text: 
 Every effort moves you 
 Every day holds a
Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])


In [10]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


In [11]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


In [12]:
total_params_gpt2 =  total_params - sum(p.numel() for p in model.out_head.parameters())
print(f"Number of trainable parameters considering weight tying: {total_params_gpt2:,}")

Number of trainable parameters considering weight tying: 124,412,160


In [13]:
# Calculate the total size in bytes (assuming float32, 4 bytes per parameter)
total_size_bytes = total_params * 4

# Convert to megabytes
total_size_mb = total_size_bytes / (1024 * 1024)

print(f"Total size of the model: {total_size_mb:.2f} MB")

Total size of the model: 621.83 MB


## decode gpt output to obtain text

In [15]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


start_context = "Hello, I am"
print(f"the input text >>> {start_context}")

encoded = tokenizer.encode(start_context)
print("encoded:", encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

model.eval() # disable dropout

out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=my_params.context_length
)

print("Output:", out)
print("Output length:", len(out[0]))

decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(f"after decoding then get >>> {decoded_text}")

the input text >>> Hello, I am
encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])
Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
Output length: 10
after decoding then get >>> Hello, I am Featureiman Byeswickattribute argue
